In [0]:
%tensorflow_version 2.x

In [0]:
import tensorflow as tf
import tensorflow.keras as keras

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

KeyboardInterrupt: 

In [0]:
housing = fetch_california_housing()

In [0]:
X_train_full,X_test,y_train_full,y_test = train_test_split(housing.data,housing.target)

In [0]:
X_train,X_valid,y_train,y_valid = train_test_split(X_train_full,y_train_full)

In [0]:
scaler =StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_valid = scaler.transform(X_valid)

#### Model Building

In [0]:
model = keras.models.Sequential(
    [
     keras.layers.Dense(30,activation="relu",input_shape=X_train.shape[1:]),
     keras.layers.Dense(20,activation="relu"),
     keras.layers.Dense(10,activation="relu"),
     keras.layers.Dense(1)
    ]
)

#### Model Compile

In [0]:
model.compile(
    loss="mean_squared_error",
    optimizer = "sgd"
)

### Tensorboard

In [0]:
import os
root_logdir = os.path.join(os.curdir, "my_logs")

def get_run_logdir():
  import time
  run_id = time.strftime("run_%Y_%m_$d-%H_%M_%S")
  return os.path.join(root_logdir,run_id)

run_logdir = get_run_logdir()

In [82]:
run_logdir

'./my_logs/run_2020_03_$d-05_43_13'

In [0]:
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

### Model Train with Checkpoints with Early stopping(both explicitly and Implicitly)

In [0]:
checkpoint_cb = keras.callbacks.ModelCheckpoint("my_first_checkpoint.h5", save_best_only=True)
# save_best_only - can be used to if and only if training has validation set. It is a best way for early stopping incase of overfitting.

In [0]:
earlyStoppingPoint_cb = keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)
#Early stopping explicitly, it saves time and wasting of resources by stopping training during overfitting

In [86]:
history = model.fit(X_train,y_train,epochs=100,validation_data=(X_valid,y_valid),callbacks=[checkpoint_cb,earlyStoppingPoint_cb,tensorboard_cb])

Train on 11610 samples, validate on 3870 samples
Epoch 1/100
11610/11610 [==============================] - 1s 83us/sample - loss: 0.8004 - val_loss: 0.5760
Epoch 2/100
11610/11610 [==============================] - 1s 63us/sample - loss: 0.5072 - val_loss: 0.4898
Epoch 3/100
11610/11610 [==============================] - 1s 60us/sample - loss: 0.4541 - val_loss: 0.4520
Epoch 4/100
11610/11610 [==============================] - 1s 61us/sample - loss: 0.4270 - val_loss: 0.4354
Epoch 5/100
11610/11610 [==============================] - 1s 62us/sample - loss: 0.4098 - val_loss: 0.4239
Epoch 6/100
11610/11610 [==============================] - 1s 65us/sample - loss: 0.4000 - val_loss: 0.4153
Epoch 7/100
11610/11610 [==============================] - 1s 67us/sample - loss: 0.3924 - val_loss: 0.4135
Epoch 8/100
11610/11610 [==============================] - 1s 68us/sample - loss: 0.3840 - val_loss: 0.4098
Epoch 9/100
11610/11610 [==============================] - 1s 66us/sample - loss: 0.377

#### Evaluation

In [87]:
model.evaluate(X_test,y_test)

5160/5160 [==============================] - 0s 29us/sample - loss: 0.2861


0.28610965122086135

#### Predictions

In [88]:
X_new = X_test[:3]
y_pred = model.predict(X_new)
y_pred

array([[3.0930493],
       [3.893079 ],
       [1.2880249]], dtype=float32)

In [89]:
y_test[:3]

array([3.138, 4.113, 1.015])

In [0]:
#!tensorboard --logdir=run_logdir --port=6006

#### Sklearn Hyperparameter Tuning

In [0]:
def build_model(n_hidden = 1, n_neurons = 30, learning_rate = 3e-3, input_shape=X_train.shape[1:]):
  model = keras.models.Sequential()
  model.add(keras.layers.InputLayer(input_shape=input_shape))
  for i in range(n_hidden):
    model.add(keras.layers.Dense(n_neurons,activation="relu"))
  model.add(keras.layers.Dense(1))
  model.compile(loss='mse',optimizer=keras.optimizers.SGD(learning_rate=learning_rate))
  return model

In [0]:
keras_sk_wrap = keras.wrappers.scikit_learn.KerasRegressor(build_model) #Wrapped around the keras model

##### Model Training

In [0]:
#keras_sk_wrap.fit(X_train,y_train,epochs=100,validation_data=(X_valid,y_valid),callbacks=[checkpoint_cb,earlyStoppingPoint_cb,tensorboard_cb])

##### Model Tuning

In [0]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal

In [0]:
param = {
    "n_hidden": [0,1,2,3],
    "n_neurons": range(1,100,10),
    "learning_rate": reciprocal(3e-4,3e-2)
}

In [0]:
rnd_sear_cv = RandomizedSearchCV(keras_sk_wrap,param,n_iter=10,cv=3)

In [97]:
rnd_sear_cv.fit(X_train,y_train,epochs=100,validation_data=(X_valid,y_valid))

Streaming output truncated to the last 5000 lines.
Epoch 26/100
7740/7740 [==============================] - 0s 61us/sample - loss: 0.3297 - val_loss: 0.3440
Epoch 27/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.3282 - val_loss: 0.3563
Epoch 28/100
7740/7740 [==============================] - 0s 56us/sample - loss: 0.3233 - val_loss: 0.3368
Epoch 29/100
7740/7740 [==============================] - 0s 57us/sample - loss: 0.3190 - val_loss: 0.3424
Epoch 30/100
7740/7740 [==============================] - 0s 56us/sample - loss: 0.3199 - val_loss: 0.3334
Epoch 31/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.3203 - val_loss: 0.3374
Epoch 32/100
7740/7740 [==============================] - 0s 58us/sample - loss: 0.3193 - val_loss: 0.3371
Epoch 33/100
7740/7740 [==============================] - 0s 56us/sample - loss: 0.3202 - val_loss: 0.3332
Epoch 34/100
7740/7740 [==============================] - 0s 57us/sample - loss: 0.3174 - val

RuntimeError: ignored

In [103]:
rnd_sear_cv.best_estimator_.model

AttributeError: ignored